# Extração e Engenharia de Features

## Objetivo
Transformar dados genômicos brutos em features numéricas utilizáveis por algoritmos de ML, normalizando para o formato TPM compatível com a matriz de treinamento.

## Passos do Pipeline
1. Extração de Features Básicas (TPM dos organismos)
2. Engenharia de Features Avançadas (diversidade, razões, mutações)
3. Normalização e Transformação
4. Alinhamento com Matriz de Treinamento

In [1]:
# Configuração inicial
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Obter diretório raiz do projeto
notebook_dir = Path().resolve()
if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent
else:
    project_root = Path('/Users/larissa/Desktop/TCC_metatrascriptomica')

# Adicionar scripts ao path
sys.path.insert(0, str(project_root))

# Verificar se scripts existe
scripts_dir = project_root / "scripts"
if not scripts_dir.exists():
    raise FileNotFoundError(
        f"Diretório 'scripts' não encontrado em {project_root}.\n"
        f"Diretório atual de trabalho: {Path().resolve()}\n"
        f"Tente executar: os.chdir('{project_root / 'notebooks'}')"
    )

# Configurar caminhos relativos
DATA_DIR = project_root / "data"
RESULTS_DIR = project_root / "results"
FEATURES_DIR = RESULTS_DIR / "features"

FEATURES_DIR.mkdir(parents=True, exist_ok=True)

# Mudar para diretório do notebook (opcional, mas útil)
os.chdir(project_root / "notebooks")

print(f"Project root: {project_root}")
print(f"Scripts directory: {scripts_dir} {'✅' if scripts_dir.exists() else '❌'}")
print(f"Diretório de trabalho: {os.getcwd()}")

Project root: /Users/larissa/Desktop/TCC_metatrascriptomica
Scripts directory: /Users/larissa/Desktop/TCC_metatrascriptomica/scripts ✅
Diretório de trabalho: /Users/larissa/Desktop/TCC_metatrascriptomica/notebooks


## 1. Extração de Features Básicas

In [2]:
from scripts.feature_extraction import extract_tpm_from_kraken2, extract_tpm_from_bracken
from scripts.feature_engineering import (
    calculate_shannon_index,
    calculate_virus_bacteria_ratio,
    detect_key_pathogens
)

# Caminhos dos relatórios
kraken_report = str(RESULTS_DIR / "kraken2_reports" / "patient_joao_kraken2_report.txt")
bracken_report = str(RESULTS_DIR / "kraken2_reports" / "patient_joao_bracken_output.txt")

# Extrair TPM (usar Bracken se disponível, senão Kraken2)
print("Extraindo features TPM...")
if os.path.exists(bracken_report):
    print("  Usando relatório Bracken (mais preciso)")
    tpm_dict = extract_tpm_from_bracken(bracken_report)
elif os.path.exists(kraken_report):
    print("  Usando relatório Kraken2")
    tpm_dict = extract_tpm_from_kraken2(kraken_report)
else:
    print("❌ Nenhum relatório encontrado. Execute o Notebook 1 primeiro.")
    tpm_dict = {}

print(f"✅ Total de organismos com TPM: {len(tpm_dict)}")

Extraindo features TPM...
  Usando relatório Kraken2
✅ Total de organismos com TPM: 50


## 2. Engenharia de Features Avançadas

In [3]:
from scripts.feature_extraction import extract_variant_features, extract_coverage_stats

# Calcular features avançadas
print("\nCalculando features avançadas...")
if tpm_dict and len(tpm_dict) > 0:
    shannon_idx = calculate_shannon_index(tpm_dict)
    print(f"  ✅ Shannon Index: {shannon_idx:.4f}")
else:
    shannon_idx = 0.0

if os.path.exists(kraken_report):
    vbr_ratio = calculate_virus_bacteria_ratio(kraken_report)
    print(f"  ✅ Razão Vírus/Bactérias: {vbr_ratio:.4f}")
    pathogens = detect_key_pathogens(kraken_report)
    print(f"  ✅ Patógenos detectados: {sum(pathogens.values())}")
else:
    vbr_ratio = 0.0
    pathogens = {}

# Features de variantes
variants_table = str(RESULTS_DIR / "variants" / "patient_joao_variants_table.csv")
if os.path.exists(variants_table):
    variant_features = extract_variant_features(variants_table)
    print(f"  ✅ Features de variantes extraídas")
else:
    variant_features = {'total_variants': 0, 'spike_mutations': 0, 'non_synonymous': 0, 'high_impact': 0}
    print(f"  ⚠️ Tabela de variantes não encontrada: {variants_table}")

# Features de cobertura
coverage_file = str(RESULTS_DIR / "variants" / "patient_joao_coverage.txt")
if os.path.exists(coverage_file):
    coverage_stats = extract_coverage_stats(coverage_file)
    print(f"  ✅ Estatísticas de cobertura extraídas")
else:
    coverage_stats = {'mean_depth': 0, 'median_depth': 0}
    print(f"  ⚠️ Arquivo de cobertura não encontrado: {coverage_file}")

# Compilar features avançadas
advanced_features = {
    'shannon_index': shannon_idx,
    'virus_bacteria_ratio': vbr_ratio,
    'spike_mutations': variant_features.get('spike_mutations', 0),
    'viral_load': coverage_stats.get('mean_depth', 0),
    **pathogens
}

print(f"\n✅ Features avançadas compiladas: {len(advanced_features)} features")


Calculando features avançadas...
  ✅ Shannon Index: 0.2958
  ✅ Razão Vírus/Bactérias: 74882.5217
  ✅ Patógenos detectados: 1
  ✅ Features de variantes extraídas
  ⚠️ Arquivo de cobertura não encontrado: /Users/larissa/Desktop/TCC_metatrascriptomica/results/variants/patient_joao_coverage.txt

✅ Features avançadas compiladas: 10 features


## 3. Normalização e Alinhamento com Matriz de Treinamento

In [4]:
from scripts.feature_engineering import create_feature_vector, apply_log_transform

# Carregar matriz de treinamento para obter colunas
training_matrix_path = str(DATA_DIR / "training" / "pivoted-virome-organisms-atleast10tpm-species-covid-TCC-pos-2.csv")

if os.path.exists(training_matrix_path):
    print("Carregando matriz de treinamento...")
    train_matrix = pd.read_csv(training_matrix_path)
    print(f"  ✅ Matriz carregada: {train_matrix.shape[0]} amostras, {train_matrix.shape[1]} colunas")

    # Obter colunas de features (excluir colunas não-feature)
    feature_columns = [col for col in train_matrix.columns if col not in ['sample_id', 'covid_status']]
    print(f"  ✅ Colunas de features: {len(feature_columns)}")

    # Criar vetor de features alinhado
    print("\nCriando vetor de features alinhado...")
    patient_features = create_feature_vector(
        tpm_dict=tpm_dict,
        training_columns=feature_columns,
        advanced_features=advanced_features
    )
    print(f"  ✅ Vetor criado: {patient_features.shape[1]} features")

    # Aplicar transformação logarítmica
    print("Aplicando transformação logarítmica...")
    patient_features_log = apply_log_transform(patient_features)
    print("  ✅ Transformação log aplicada")

    # Salvar vetores
    features_file = str(FEATURES_DIR / "patient_joao_features_vector.csv")
    features_log_file = str(FEATURES_DIR / "patient_joao_features_vector_log.csv")
    
    patient_features.to_csv(features_file, index=False)
    patient_features_log.to_csv(features_log_file, index=False)
    
    print(f"\n✅ Vetores de features salvos:")
    print(f"  - {features_file}")
    print(f"  - {features_log_file}")
else:
    print(f"❌ Matriz de treinamento não encontrada: {training_matrix_path}")
    print("   O vetor de features não pode ser criado sem a matriz de treinamento.")

Carregando matriz de treinamento...
  ✅ Matriz carregada: 100 amostras, 4671 colunas
  ✅ Colunas de features: 4671

Criando vetor de features alinhado...
  ✅ Vetor criado: 4678 features
Aplicando transformação logarítmica...
  ✅ Transformação log aplicada

✅ Vetores de features salvos:
  - /Users/larissa/Desktop/TCC_metatrascriptomica/results/features/patient_joao_features_vector.csv
  - /Users/larissa/Desktop/TCC_metatrascriptomica/results/features/patient_joao_features_vector_log.csv
